In [1]:
# useing this variable for saving models and summaries
exp_name = "HRNN_1001_z"

## Running CNN then RNN
5 layers CNN [2, 2, 2, 2, 2], then RNN, bias_shape = [32,64,64, 64, 64]

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import sys
from hdf5storage import savemat
import pickle

sys.path.append('../../src')

In [3]:
from models.HRNN_model import Model
import training  
import evaluation.nn_eval as nn_eval 

## Loading synthetic data

In [4]:
#loading data
data_path= '../../data/synthetic2/' + 'processed4.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [5]:
train_data.shape

(900, 8192, 12)

## Model

In [6]:
## Model Parameter
n_variables= 12
learning_rate = 0.001
batch_size = 10

wavelet_levels= 6
convolution_levels = 5
pooling_stride = [2, 2, 2, 2, 2]
weights_shape = [[5,n_variables, 32],
                 [5, 32, 64],
                 [5, 64, 64],
                 [5, 64, 64],
                 [5, 64, 64]]

bias_shape = [32, 64, 64, 64, 64]
activation = tf.nn.relu
wavelet_weights_shape = [[5, n_variables, n_variables]]    #high_pass and low_pass dimensions

lstm_n_hidden = 128
n_classes = 1

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)

pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)

variables_weights = np.ones((n_classes,))

In [7]:
pos_weight

array([ 2.46045165])

In [8]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [9]:
summary_path = '../../summary_synthetic/train_gradual_test_abrupt/' + exp_name 
checkpoint_path = '../../model_synthetic/train_gradual_test_abrupt/' + exp_name + '/' + exp_name 

In [10]:
model = Model(n_variables = n_variables,
                            learning_rate = learning_rate,
                            n_classes = n_classes)

In [11]:
model.build_model(pos_weight = pos_weight, 
                 learning_rate = learning_rate,
                 wavelet_levels = wavelet_levels,    
                 convolution_levels = convolution_levels,    
                 pooling_stride = pooling_stride,     
                 weights_shape = weights_shape,
                 bias_shape = bias_shape,
                 wavelet_weights_shape = wavelet_weights_shape,    
                 activation = activation,
                 lstm_n_hidden = lstm_n_hidden,
                 trainable_wavelet = True,
                 trainable_conv = True,
                 trainable_last = True,
                 variables_weights = variables_weights,
                 optimizer = tf.train.AdamOptimizer
)

6
Tensor("Squeeze_29:0", shape=(?, ?, 64), dtype=float32)
Tensor("Squeeze_29:0", shape=(?, ?, 64), dtype=float32) Tensor("zeros:0", shape=(?, ?, 128), dtype=float32) 5
Tensor("Squeeze_24:0", shape=(?, ?, 64), dtype=float32) Tensor("Reshape:0", shape=(?, ?, ?), dtype=float32) 4
Tensor("Squeeze_19:0", shape=(?, ?, 64), dtype=float32) Tensor("Reshape_1:0", shape=(?, ?, ?), dtype=float32) 3
Tensor("Squeeze_14:0", shape=(?, ?, 64), dtype=float32) Tensor("Reshape_2:0", shape=(?, ?, ?), dtype=float32) 2
Tensor("Squeeze_9:0", shape=(?, ?, 64), dtype=float32) Tensor("Reshape_3:0", shape=(?, ?, ?), dtype=float32) 1
Tensor("Squeeze_4:0", shape=(?, ?, 64), dtype=float32) Tensor("Reshape_4:0", shape=(?, ?, ?), dtype=float32) 0
Tensor("Squeeze_30:0", shape=(?, ?, 1), dtype=float32) Tensor("Reshape_7:0", shape=(?, ?, 1), dtype=float32)


In [12]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'change')

In [13]:
trainer.train(max_iter = 10000,
                train_eval_step=100,
                validation_eval_step=100,
                display_step=100)

Training: 0
AUC = 0.247 vs. 0.266
F1 = 0.443 vs. 0.432
Precision = 0.285 vs. 0.275
Recall = 1.000 vs. 1.000
Validation: 0
AUC = 0.079 vs. 0.073
F1 = 0.141 vs. 0.137
Precision = 0.078 vs. 0.073
Recall = 0.786 vs. 0.981
Minibatch Loss= 0.979648
Training: 100
AUC = 0.773 vs. 0.274
F1 = 0.691 vs. 0.433
Precision = 0.685 vs. 0.277
Recall = 0.697 vs. 0.996
Validation: 100
AUC = 0.366 vs. 0.073
F1 = 0.417 vs. 0.137
Precision = 0.394 vs. 0.073
Recall = 0.441 vs. 0.998
Minibatch Loss= 0.687678
Training: 200
AUC = 0.883 vs. 0.294
F1 = 0.797 vs. 0.457
Precision = 0.840 vs. 0.296
Recall = 0.758 vs. 0.997
Validation: 200
AUC = 0.609 vs. 0.073
F1 = 0.594 vs. 0.137
Precision = 0.598 vs. 0.073
Recall = 0.590 vs. 0.995
Minibatch Loss= 0.684738
Training: 300
AUC = 0.919 vs. 0.275
F1 = 0.843 vs. 0.440
Precision = 0.835 vs. 0.282
Recall = 0.852 vs. 0.999
Validation: 300
AUC = 0.728 vs. 0.074
F1 = 0.684 vs. 0.137
Precision = 0.669 vs. 0.073
Recall = 0.699 vs. 0.972
Minibatch Loss= 0.506343
Training: 400
AU

In [21]:
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../../model_synthetic/train_gradual_test_abrupt/HRNN_1001_z/HRNN_1001_z-10000'

In [22]:
#saver = tf.train.Saver()
#saver.restore(sess, checkpoint_path+'-1000')

In [23]:
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

AUC = 0.804 vs. 0.073
F1 = 0.775 vs. 0.137
Precision = 0.714 vs. 0.073
Recall = 0.849 vs. 0.989


In [24]:
out = nn_eval.deploy (model, sess, test_data, test_gt)

In [25]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
out = np.squeeze(out, axis = 2)

In [26]:
out.shape

(1000, 256)

In [27]:
savemat('../../poc_synthetic/train_gradual_test_abrupt/poc'+exp_name+'.mat', {'poc':out})